## Manipulating files with bedtoools
### Bioinformatics Coffee Hour - May 18, 2020

### Genomic Intervals
Many types of bioinformatics data are stored as genomic intervals, that is ranges (potentially discontinuous) along a sequence. Of course this includes things like gene models, but also the output of peak callers, enhancer predictions, and many other things. This genomic interval information is typically stored in either GFF or bed files. Today we are going to focus on bed files. 

Bed files use a 0-based start (first base of a chromosome is 0), and a 1-based end. This means the interval from the first base to the hundredth base of a chromosome would be represented as start = 0, end = 100 in a bed file. Another way to think about this is that bed files are 0-based, half-open (the start is included in the interval but not the end). Note that GFF files use a different convention, just to make things more confusing. 

Bed files are just tab-delimited files with a fixed set of fields: 

|col|field|meaning|
|:---|:---|:--|
|1|seqname|chr or scaffold id interval is on|
|2|start|start position (0-based) of interval|
|3|end|end position (1-based) of interval|
|4|name|name of interval|
|5|score|score, e.g. for peak calls or other intervals with scores; can be .|
|6|strand|strand interval is on for strand-specific intervals|
|7|thickStart|graphical parameter for genome browser display|
|8|thickEnd|graphical parameter for genome browser display|
|9|itermRgb|graphical parameter for genome browser display|
|10|blockCount|for multi-exon genes and similar|
|11|blockSizes|for multi-exon genes and similar|
|12|blockStarts| for multi-exon genes and similar|

Only the first three fields are required, but you cannot skip fields (so if you want to include strand, you need to also include name and score in that order). This means that you can manipulate bed files with standard Unix tools (such as awk that we introduced a few weeks ago). However, many operations on intervals can get quite complicated, and so there are specialized tools for dealing with this specific kind of data. One of the most popular and useful is bedtools, which will be the focus of our demo today.

First, we'll just look at an example bed file:

In [ ]:
head data/human_enhancers_chr22.bed

### Finding the closest gene to something

One very common task in interval manipulation is trying to find the closest interval from one file to intervals in another file, e.g. you have a list of putative ehnancers (as a bed file) and you want to know the closest gene to each one. In bedtools, this can be done very simply with the `closest` option:

In [ ]:
bedtools closest -a data/human_enhancers_chr22.bed -b data/ucscGenes_chr22.bed | head

Note we pipe the output to head with the `|` operator so that we only see the first few lines of the output.

Many bedtools commands for comparing things use `-a` and `-b` to indicate the two files to compare. Although the exact meaning varies, usually this can be read as "for each element in a, compare to all elements in b." So in this case, we get the closest gene to each enhancer. We could also do this in reverse, to get the closest enhancer for each gene.

In [ ]:
bedtools closest -a data/ucscGenes_chr22.bed -b data/human_enhancers_chr22.bed | head

Notice that each element in the `-a` file only appears once, but elements in `-b` can appear many times. 

We might want to list not just the closest element, but how far away it is. We can do this using the `-d` option to bedtools closest, like so:

In [ ]:
bedtools closest -a data/human_enhancers_chr22.bed -b data/ucscGenes_chr22.bed -d | head

Now, the last column is the distance between the element in the A file and the element in the B file. Notice that some of these distances are 0, which means that the closest element in B overlaps the element in A. We might want to find the closest non-overlapping element, instead of the closest element. We can do this with the `-io` option:

In [ ]:
bedtools closest -a data/human_enhancers_chr22.bed -b data/ucscGenes_chr22.bed -d -io | head

Like most bedtools tools, `closest` has a number of additional options to control filtering, reporting, and what is considered closest (e.g., requiring the same or different strand, or looking only at upstream or downstream features). You can see all the options with `-h` to display the help:

In [ ]:
bedtools closest -h

As an exercise, look through the help display and think about how you would find the closest **upstream** enhancer to each gene.

### Finding overlaps

Another common task is finding overlaps between two sets of intervals. For example, we might want to know which of our enhancers identified in neural crest cells are shared with enhancers identified in some other cell type. In your directory you should see another bed file, H1-hESC-H3K27Ac_chr22.bed, which has H3K27Ac peaks from human embryonic stem cells. We'll use bedtools to get the overlaps between this file and our human enhancers file.

Intersect can be a little tricky to sort out the options, so we'll use this figure as reference:

<div style='height: 400px; overflow: hidden'><img src='https://raw.githubusercontent.com/arq5x/bedtools2/v2.29.2/docs/content/images/tool-glyphs/intersect-glyph.png'></div>

Like `closest`, `bedtools intersect` uses a `-a` and a `-b` file. Each feature in A is compared to all features in B to identify overlaps. The simplest option is:

In [ ]:
bedtools intersect -a data/human_enhancers_chr22.bed -b data/H1-hESC-H3K27Ac_chr22.bed | head

Note that this only lists the features in `A` that have an overlap in `B`, and it doesn't tell us what they overlap. If we wanted to list all features in `A` and indicate what they overlap, we could use the `-wao` option:

In [ ]:
bedtools intersect -a data/human_enhancers_chr22.bed -b data/H1-hESC-H3K27Ac_chr22.bed -wao | head

This output is a little complicated, so let's walk through it. The first 5 columns are the elements in A, and would look the same as `head data/human_enhancers_chr22.bed`. Next we have the element in B that overlaps, or `.` if no elements overlap. This is also a five-column bed file, ending with a score that is between 0 and 1000. Finally, we have an additional column that is the number of base pairs that overlap. Note that in this case, most of the elements in B are much bigger than the elements in A, so the overlap is typically the length of the element in A.

Let's say now we want to split our human_enhancers file into two separate files, one for those that overlap a H1-hESC-H3K27Ac and one for those that don't. We could do this using `awk` or `grep` on the output we just produced, but we can also do this with bedtools itself. There is an option, `-wa` that writes the original entry in A if it overlaps any entry in B, and other option `-v` that only reports elements in A with no overlap in B:

In [ ]:
bedtools intersect -a data/human_enhancers_chr22.bed -b data/H1-hESC-H3K27Ac_chr22.bed -wa -v | head

In [ ]:
bedtools intersect -a data/human_enhancers_chr22.bed -b data/H1-hESC-H3K27Ac_chr22.bed -wa | head

Bedtools intersect has **lots** of options, and usually there is a way to do anything that involves comparing to sets of intervals and finding overlaps with it.

### Computing promoter intervals 

So far we've looked at examples of comparing two different files, but some bedtools options work with a single bed file. An example is the `flank` command, which creates a new interval adjacent to existing intervals. We can use this to, for example, compute promoter intervals for each gene, where we define promoters as the 2kb upstream of the start site:

In [ ]:
bedtools flank -i data/ucscGenes_chr22.bed -g data/hg38.genome -l 2000 -r 0 -s | head

One of the powerful features of bedtools is that you can combine different tools with the Unix `|`, to allow the output of one tool to be the input to another. We can use this to identify all the enhancers that fall within promoter regions of genes:

In [ ]:
bedtools flank -i data/ucscGenes_chr22.bed -g data/hg38.genome -l 2000 -r 0 -s | bedtools intersect -a data/human_enhancers_chr22.bed -b - -u | head

### Quick hits

There are many other possibilities for what bedtools can do. We'll end with just a couple of more examples of how to solve common problems with bedtools.

#### Make a bed file of intergenic regions from a bed file of genes

In [ ]:
bedtools complement -i data/ucscGenes_chr22.bed -g data/hg38.genome | head

#### Randomly shuffle the location of enhancers, avoiding placing them in genes

In [ ]:
bedtools shuffle -i data/human_enhancers_chr22.bed -g data/hg38.genome -excl data/ucscGenes_chr22.bed | head

#### Compute a single metric to measure similarity between two sets of intervals

In [ ]:
bedtools flank -i data/ucscGenes_chr22.bed -g data/hg38.genome -l 2000 -r 0 -s | bedtools sort -i - | bedtools jaccard -a data/H1-hESC-H3K27Ac_chr22.bed -b -

In [ ]:
bedtools random -n 743 -l 2000 -g data/hg38.genome | bedtools sort -i - | bedtools jaccard -a data/H1-hESC-H3K27Ac_chr22.bed -b -